Generating code for a model:

In [1]:
import pytorch_composer
import pytorch_composer.datasets


# A random sequence of neural network layers. Any positive integer shoud be a valid dimension arguement:
sequence = [
    ["Conv2d", 6],
    ["MaxPool2d", 2],
    ["Linear", 16],
    ["Relu"],
    ["MaxPool2d", 2],
    ["Linear",43],
    ["RNN",12],
    ["MaxPool2d", 2],
    ["Relu"],
    ["Flat"],
    ["Linear",38],
]
dataset = pytorch_composer.datasets.CIFAR10()
model = pytorch_composer.Model(sequence, dataset)
loop = pytorch_composer.Classifier(model)

code = pytorch_composer.Code([dataset,model,loop])

# The code can be saved in a text file with:
#     code.save()

code

import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torch.optim as optim
import torch


# Load and normalize the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):


Using the generated code:

In [2]:
code.execute()

Files already downloaded and verified
Files already downloaded and verified
[1,  2000] loss: 2.302
[1,  4000] loss: 2.240
[1,  6000] loss: 2.065
[1,  8000] loss: 1.990
[1, 10000] loss: 1.922
[1, 12000] loss: 1.834
[2,  2000] loss: 1.751
[2,  4000] loss: 1.703
[2,  6000] loss: 1.666
[2,  8000] loss: 1.617
[2, 10000] loss: 1.605
[2, 12000] loss: 1.573
Finished Training
